# Subgradient method on MILP

In [9]:
import sys
sys.path.append('../sddip')

In [10]:
import gurobipy as gp
import numpy as np
import pandas as pd

import sddip.dualsolver as ds

## Problem parameters

In [11]:
i_max = 4
j_max = 5

d = np.array([120, 150, 100, 150, 180])
a = np.array([250, 300, 400, 700])
k = np.array([700, 900, 300, 400])
c = np.array([[2, 1, 3, 4, 5],[4, 2, 8, 14, 10], [4, 12, 3, 14, 7], [8, 17, 7, 9, 15]])

x = np.empty((i_max, j_max), dtype=object)
y = np.empty(i_max, dtype=object)

## Original problem

In [12]:
m = gp.Model("Standortproblem")


for i in range(i_max):
    y[i] = m.addVar(vtype = gp.GRB.BINARY, name = "y_%i"%(i+1))
    for j in range(j_max):
        for j in range(j_max):
            x[i,j] = m.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "x_%i%i"%(i+1,j+1))

m.update()

expr_o1 = gp.LinExpr(k, y)
expr_o2 = gp.LinExpr(c.flatten(), x.flatten())
obj = expr_o1+expr_o2
m.setObjective(obj)

m.addConstrs(gp.quicksum(x[:,j]) == d[j] for j in range(j_max))
m.addConstrs(gp.quicksum(x[i,:]) <= a[i]*y[i] for i in range(i_max))

m.update()

m.optimize()
m.printAttr("X")



####################################
# Solution
####################################
print('#################################################')

# Optimal value
v = m.getObjective().getValue()
print("Optimal value v= {}".format(v))

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 9 rows, 104 columns and 44 nonzeros
Model fingerprint: 0x6d512bd2
Variable types: 100 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 2e+02]
Presolve removed 0 rows and 80 columns
Presolve time: 0.00s
Presolved: 9 rows, 24 columns, 44 nonzeros
Variable types: 20 continuous, 4 integer (4 binary)
Found heuristic solution: objective 4640.0000000

Root relaxation: objective 3.990000e+03, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3990.00000    0    1 4640.00000 3990.00000  14.0%     -    0s
H    0     0                    4440.0000000 399

## Lagrangian Dual

In [13]:
m = gp.Model("Standortproblem")


for i in range(i_max):
    y[i] = m.addVar(vtype = gp.GRB.BINARY, name = "y_%i"%(i+1))
    for j in range(j_max):
        for j in range(j_max):
            x[i,j] = m.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = "x_%i%i"%(i+1,j+1))

expr_o1 = gp.LinExpr(k, y)
expr_o2 = gp.LinExpr(c.flatten(), x.flatten())

objective_terms = expr_o1+expr_o2
relaxed_terms = [gp.quicksum(x[:,j]) - d[j] for j in range(j_max)]

#m.setObjective(objective_terms)
#m.addConstrs(quicksum(x[:,j]) == d[j] for j in range(j_max))
m.addConstrs(gp.quicksum(x[i,:]) <= a[i]*y[i] for i in range(i_max))

m.update()

m.setParam("Outputflag",0)

### Subgradient mehod

In [14]:
sg_method = ds.SubgradientMethod(max_iterations = 50)
sg_method.output_flag = True

rlx_model, results = sg_method.solve(m, objective_terms, relaxed_terms, 5000)

Subgradient Method started
New lower bound
Iteration: 0 | Optimal value: 0.0 | Gradient magnitude: 319.06112267087633 | Step size: 0.0031341957040361135
New lower bound
Iteration: 1 | Optimal value: 319.06112267087633 | Gradient magnitude: 319.06112267087633 | Step size: 0.0031341957040361135
New lower bound
Iteration: 2 | Optimal value: 638.1222453417527 | Gradient magnitude: 319.06112267087633 | Step size: 0.0031341957040361135
New lower bound
Iteration: 3 | Optimal value: 957.183368012629 | Gradient magnitude: 319.06112267087633 | Step size: 0.0031341957040361135
New lower bound
Iteration: 4 | Optimal value: 1276.2444906835053 | Gradient magnitude: 319.06112267087633 | Step size: 0.0031341957040361135
New lower bound
Iteration: 5 | Optimal value: 1595.3056133543819 | Gradient magnitude: 319.06112267087633 | Step size: 0.0031341957040361135
New lower bound
Iteration: 6 | Optimal value: 1914.366736025258 | Gradient magnitude: 319.06112267087633 | Step size: 0.0031341957040361135
New l

In [15]:
print(f"Best lower bound: {results.obj_value}")
print(f"Best multipliers: {results.multipliers}")

Best lower bound: 3911.9949389260664
Best multipliers: [-4.98328698 -4.52984567 -4.07119798 -7.77016022 -7.96055037]


In [16]:
m.setParam("Outputflag",1)
rlx_model.printAttr("X")

Set parameter OutputFlag to value 1

    Variable            X 
-------------------------
         y_1            1 
        x_14          250 
         y_3            1 
        x_31          400 
